In [1]:
ours2orig = [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27,
28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 254, 41, 42, 43, 44, 45,
46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64,
65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 85,
86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103,
104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118,
119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 136, 138, 132,
133, 134, 135, 220, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152,
153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167,
168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 185, 186,
187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201,
202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216,
217, 218, 219, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232,
233, 137, 234, 235, 236, 237, 238, 239, 240, 241, 250, 251, 255, 256, 257,
258, 259, 260, 261, 262, 263, 242, 243, 244, 245, 0, 1, 2, 3, 4, 5, 6, 7, 8,
9, 10, 11, 83, 84, 131, 139, 140, 141, 181, 182, 183, 184, 246, 247, 248,
249, 252, 253]

def vec2mat(v):
    a,b = np.triu_indices(264,1)
    m = np.zeros((264,264))
    m[a,b] = v
    return m+m.T

def remap(fc, roimap=ours2orig):
    fc = fc[roimap,:]
    fc = fc[:,roimap]
    return fc

print('Complete')

Complete


In [2]:
# Load real PNC data
# Must remap!

import pickle
import numpy as np

pncdir = '/home/anton/Documents/Tulane/Research/data-sav/anton/cohorts/PNC/'
pncdemo = pickle.load(open(f'{pncdir}/demographics.pkl', 'rb'))

rest = []
nback = []
emoid = []
restmat = []
nbackmat = []
emoidmat = []
race = []
sex = []
age = []

a,b = np.triu_indices(264,1)

for sub in pncdemo['age_at_cnb']:
    try:
        ra = pncdemo['Race'][sub]
        ag = pncdemo['age_at_cnb'][sub]
        se = pncdemo['Sex'][sub]
        if ra not in ['AA', 'EA']:
            continue
        ra = ra == 'AA'
        se = se == 'M'
        r = np.load(f'{pncdir}/fc/{sub}_task-rest_fc.npy')
        n = np.load(f'{pncdir}/fc/{sub}_task-nback_fc.npy')
        e = np.load(f'{pncdir}/fc/{sub}_task-emoid_fc.npy')
        r = remap(vec2mat(r))
        n = remap(vec2mat(n))
        e = remap(vec2mat(e))
        race.append(ra)
        sex.append(se)
        age.append(ag)
        rest.append(r[a,b])
        nback.append(n[a,b])
        emoid.append(e[a,b])
        restmat.append(r)
        nbackmat.append(n)
        emoidmat.append(e)
    except:
        pass

rest = np.stack(rest)
nback = np.stack(nback)
emoid = np.stack(emoid)
restmat = np.stack(restmat)
nbackmat = np.stack(nbackmat)
emoidmat = np.stack(emoidmat)
race = np.array(race).astype('int')
sex = np.array(sex).astype('int')
age = np.array(age)

print([a.shape for a in [rest, nback, emoid, restmat, nbackmat, emoidmat, race, sex, age]])

[(1193, 34716), (1193, 34716), (1193, 34716), (1193, 264, 264), (1193, 264, 264), (1193, 264, 264), (1193,), (1193,), (1193,)]


In [4]:
# Load fake data

import pickle

fakemats = pickle.load(open('/home/anton/Documents/Tulane/Research/Work/Imputation/fake_rest200_5.pkl', 'rb'))
fakemats = np.stack(fakemats)

print(fakemats.shape)

(939, 34716)


In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.model_selection import train_test_split

class D(nn.Module):
    def __init__(self):
        super(D, self).__init__()
        self.fc1 = nn.Linear(34716, 1).float().cuda()

    def forward(self, x):
        x = self.fc1(x).squeeze()
        x = torch.stack([-x, x], dim=1)
        return x

x1tr, x1t, x2tr, x2t, x3tr, x3t, xftr, xft = train_test_split(rest[:len(fakemats)], 
                                                              nback[:len(fakemats)], emoid[:len(fakemats)], fakemats, train_size=500)

xtr = np.concatenate([x1tr, x2tr, x3tr, xftr])
xt = np.concatenate([x1t, x2t, x3t, xft])

ytr = np.concatenate([np.zeros(3*len(x1tr)), np.ones(len(x2tr))])
yt = np.concatenate([np.zeros(3*len(x1t)), np.ones(len(x2t))])

xtr = torch.from_numpy(xtr).float().cuda()
xt = torch.from_numpy(xt).float().cuda()

ytr = torch.from_numpy(ytr).long().cuda()

d = D()
optim = torch.optim.Adam(d.parameters(), lr=1e-4, weight_decay=1e-4)
ce = nn.CrossEntropyLoss()

nepochs = 200
pperiod = 5

for e in range(nepochs):
    optim.zero_grad()
    yhat = d(xtr)
    loss = ce(yhat, ytr)
    loss.backward()
    optim.step()
    if e % pperiod == 0 or e == nepochs-1:
        print(f'{e} {loss}')

print('Done')

with torch.no_grad():
    yhat = d(xt)[:,0]
    print(yhat)

0 0.6573747396469116
5 0.46607911586761475
10 0.3215673863887787
15 0.20779338479042053
20 0.16103608906269073
25 0.11550205945968628
30 0.09040851891040802
35 0.07172330468893051
40 0.059972379356622696
45 0.05029452219605446
50 0.04420534893870354
55 0.03890532627701759
60 0.03496279567480087
65 0.031778618693351746
70 0.0290488563477993
75 0.026804301887750626
80 0.024862956255674362
85 0.023158736526966095
90 0.02166961506009102
95 0.02034374512732029
100 0.01915321685373783
105 0.01808193325996399
110 0.017110975459218025
115 0.016225969418883324
120 0.015416857786476612
125 0.014674432575702667
130 0.013990703970193863
135 0.013359250500798225
140 0.01277456060051918
145 0.012231796979904175
150 0.011726760305464268
155 0.011255842633545399
160 0.01081585418432951
165 0.010403988882899284
170 0.010017753578722477
175 0.009654953144490719
180 0.009313617832958698
185 0.008991996757686138
190 0.008688516914844513
195 0.008401758037507534
199 0.008183536119759083
Done
tensor([ 2.450

In [13]:
d.fc1.weight

Parameter containing:
tensor([[ 2.5861e-03,  6.9773e-04, -3.5670e-03,  ..., -3.6303e-03,
         -7.0966e-03,  3.9245e-03],
        [ 9.0609e-03,  3.8559e-03,  4.4965e-03,  ..., -2.0645e-03,
          1.4544e-03, -2.2451e-03],
        [-1.2576e-03, -2.8317e-03, -3.1604e-03,  ..., -1.7655e-03,
          2.6367e-03, -8.8082e-05],
        ...,
        [ 8.6080e-03,  1.4550e-03,  6.4812e-03,  ..., -1.8886e-03,
          2.4047e-03, -6.5631e-03],
        [ 1.9330e-03, -1.1343e-03,  2.7218e-03,  ..., -1.6337e-03,
         -1.3947e-03, -1.7534e-03],
        [-3.8930e-03, -1.4396e-03, -7.8373e-04,  ..., -1.7212e-03,
          9.2548e-05, -5.4424e-04]], device='cuda:0', requires_grad=True)